In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip
!pip install unsloth
!pip install datasets  # for loading MRPC
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 179.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 32.7 MB/s eta 0

In [ ]:
from datasets import load_dataset

# Load MRPC (train + validation + test splits)
mrpc_data = load_dataset("glue", "mrpc")
train_data = mrpc_data["train"]
val_data   = mrpc_data["validation"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
def format_example(ex):
    label_text = "Yes" if ex["label"] == 1 else "No"
    text = (
        "/no_think Given these two sentences, are they paraphrases?\n"
        f"Sentence1: {ex['sentence1']}\n"
        f"Sentence2: {ex['sentence2']}\n"
        f"Paraphrase: {label_text}. "
    )
    return {"text": text}

train_data = train_data.map(format_example)
val_data   = val_data.map(format_example)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0,
 'text': '/no_think Given these two sentences, are they paraphrases?\nSentence1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .\nSentence2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .\nParaphrase: Yes. '}

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen3-1.7B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=False  # or True/False depending on memory; 4-bit will use less memory
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0
)


Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.7 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="qwen3_mrpc",
    per_device_train_batch_size=4,   # adjust based on GPU memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,   # to simulate larger batch if needed
    logging_steps=10,
    weight_decay=0.01,
    learning_rate=3e-4,
    num_train_epochs=10,

)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=val_data,
    dataset_text_field="text",
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    args=training_args,
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3668 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
!nvidia-smi
!pip install --upgrade triton
!export TRITON_DISABLE_LINE_INFO=1
!export WANDB_API_KEY=7e1e7b39623315f0210e0ef63f31f9df81f324ae

Sun May 11 08:47:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P0             29W /   70W |    3504MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!export WANDB_API_KEY=7e1e7b39623315f0210e0ef63f31f9df81f324ae

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,668 | Num Epochs = 10 | Total steps = 9,170
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,422,528/1,726,997,504 (0.37% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jl13122 (jl13122-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.431300
20,1.667900
30,1.576500
40,1.561500
50,1.507200
60,1.480200
70,1.591100
80,1.517900
90,1.506000
100,1.399500


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=9170, training_loss=0.585370668720133, metrics={'train_runtime': 3546.6446, 'train_samples_per_second': 10.342, 'train_steps_per_second': 2.586, 'total_flos': 2.9290098712633344e+16, 'train_loss': 0.585370668720133})

In [ ]:
import torch

model.eval()
predictions, references = [], []
i = 0
for ex in val_data:
    if i == 10: break
    prompt = ex["text"].rsplit("Paraphrase:", 1)[0] + "Paraphrase:"  # remove label
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    print("the prompt is:", prompt, "\n", "the inputs are:", inputs)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=5)

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_token = decoded.lower().split("paraphrase:")[-1].strip().split()[0]

    predictions.append(1 if pred_token.startswith("yes") else 0)
    print("the outputs are:", outputs, "\n", "the decoded is:", decoded, "\n", "the pred_token is:", pred_token)
    references.append(ex["label"])
    i+=1


the prompt is: /no_think Given these two sentences, are they paraphrases?
Sentence1: He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
Sentence2: " The foodservice pie business does not fit our long-term growth strategy .
Paraphrase: 
 the inputs are: {'input_ids': tensor([[33100,  5854,   766, 16246,  1493,  1378, 22870,    11,   525,   807,
         62230, 26565,  5267, 84564,    16,    25,  1260,  1053,   279,  3607,
          7936,  4349,  2562,  3171,   364,    83,  4946,   279,  2813,   364,
            82,  1293,  9663,  6513,  8282, 16448, 84564,    17,    25,   330,
           576,  3607,  7936,  4349,  2562,  1558,   537,  4946,  1039,  1293,
          9663,  6513,  8282, 16448,  4272,  1342, 10632,    25]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
result = metric.compute(predictions=predictions, references=references)
print(result)


{'accuracy': 0.9, 'f1': 0.9230769230769231}
